## Key generation and encryption

In [1]:
q = 131
n = 128
k = n // 2
min_dist = n - k + 1
num_eras = min_dist // 3
t = (min_dist - num_eras - 1) // 2

F.<z> = GF(q)
Fstar = [i for i in F if i != 0]
X = sample(Fstar, n)
Y = sample(Fstar, n)
C = codes.GeneralizedReedSolomonCode(X, k, Y)
G = C.generator_matrix()

M = random_matrix(F, n)
while M.rank() != n:
    M = random_matrix(F, n)
eras_pos = sample(range(n), num_eras)
R = zero_matrix(F, n)
R[:, eras_pos] = random_matrix(F, n, num_eras)

def random_permutation_matrix(n):
    perm = Permutations(n).random_element()
    M = matrix(F, n, n)
    for i in range(n):
        M[i, perm(i+1)-1] = F(1)
    return M

Gpub = (G*M).rref()
P = random_permutation_matrix(n)
Epub = (random_matrix(F, n, k) * G + R + P)*M

# encryption
def gen_error(n, t):
    res = zero_vector(F, n)
    for i in sample(range(n), t):
        res[i] = choice(Fstar)
    return res

msg = random_vector(F, k)
e = gen_error(n, t)
y_enc = msg * Gpub + e*Epub

# The attack

In [2]:
def square(A):
    N = A.nrows()
    res = zero_matrix(F, N*(N-1)//2 + N, A.ncols())
    r = 0
    for i in range(N):
        for j in range(i, N):
            res[r] = A[i].pairwise_product(A[j])
            r += 1
    return res

def remove_rand_columns_pass(A):
    if square(A).rank() == A.ncols():
        ns = 1
        B = codes.LinearCode(A).shortened([i for i in range(A.ncols() - ns, A.ncols())]).generator_matrix()
        while square(B).rank() == B.ncols():
            ns += 1
            B = codes.LinearCode(A).shortened([i for i in range(A.ncols() - ns, A.ncols())]).generator_matrix()
        rr = square(B).rank()
        rnd_ind = []
        for i in range(0, A.ncols() - ns - 1):
            D = codes.LinearCode(A).shortened([i for i in range(A.ncols() - ns, A.ncols())] + [i]).generator_matrix()
            if square(D).rank() == rr - 1:
                rnd_ind.append(i)
        AA = A.delete_columns(rnd_ind)
        ns = 0
        B = codes.LinearCode(AA).shortened([i for i in range(ns)]).generator_matrix()
        while square(B).rank() == B.ncols():
            ns += 1
            B = codes.LinearCode(AA).shortened([i for i in range(ns)]).generator_matrix()
        rr = square(B).rank()
        rnd_ind = []
        for i in range(ns, AA.ncols()):
            D = codes.LinearCode(AA).shortened([i for i in range(ns)] + [i]).generator_matrix()
            if square(D).rank() == rr - 1:
                rnd_ind.append(i)
        return AA.delete_columns(rnd_ind)
    else:
        rr = square(A).rank()
        rnd_ind = []
        for i in range(A.ncols()):
            if square(A.delete_columns([i])).rank() == rr-1:
                rnd_ind.append(i)
        return A.delete_columns(rnd_ind)

def remove_rand_columns(A):
    AA = remove_rand_columns_pass(A)
    return remove_rand_columns_pass(AA)

def get_punctured_indices(original_matrix, punctured_matrix):
    original_cols = original_matrix.columns()
    punctured_cols = punctured_matrix.columns()
    punctured_indices = []
    for i, col in enumerate(original_cols):
        if col not in punctured_cols:
            punctured_indices.append(i)    
    return punctured_indices

def recover_GRS_support(G, x0, x1, xk):
    n = G.ncols()
    k = G.nrows()
    G1 = G.rref()
    x = [q+1]*n
    x[0], x[1], x[k] = x0, x1, xk
    count = 3
    i = 0; i_ = 1; j_ = k
    for j in range(k+1, n):
        gamma = (G1[i,j] * G1[i_, j_]) / (G1[i, j_] * G1[i_, j])
        denom = (x[j_] - x[i]) - gamma*(x[j_] - x[i_])
        if denom != 0:
            xj = (x[i_]*(x[j_]-x[i]) - gamma*x[i]*(x[j_] - x[i_])) / denom
            if xj in x:
                return x, False
            x[j] = xj
        else:
            return x, False
    i_ = 0; j = k; j_ = k+1
    for i in range(2, k):
        gamma = (G1[i,j] * G1[i_, j_]) / (G1[i, j_] * G1[i_, j])
        denom = gamma*(x[j_] - x[i_]) - (x[j] - x[i_])
        if denom != 0:
            xi = (gamma*x[j]*(x[j_]-x[i_]) - (x[j] - x[i_])*x[j_]) / denom
            if xi in x:
                return x, False
            x[i] = xi
        else:
            return x, False            
    return x, True

def recover_GRS_multiplier(G, new_x, k):
    n_ = len(new_x)
    In = identity_matrix(F, n_)
    H = codes.GeneralizedReedSolomonCode(new_x, k).parity_check_matrix()
    T = matrix([(G * diagonal_matrix(i) * H.T).list() for i in In])
    sol = T.left_kernel()
    #print(sol.dimension())
    b = sol.random_element()
    while 0 in b:
        b = sol.random_element()
    new_y = vector([i^-1 for i in b])
    return new_y

In [3]:
Hpub = codes.LinearCode(Gpub).parity_check_matrix()

H1 = Hpub * (Epub.T)
G1 = codes.LinearCode(H1).parity_check_matrix()

In [4]:
# Removing non-GRS columns in the resulting scheme

G2 = remove_rand_columns(G1)
punctured_indices = get_punctured_indices(G1, G2)

G1.ncols(), G2.ncols(), G1.ncols() - G2.ncols() == num_eras

(128, 107, True)

In [5]:
F_list = F.list()
j = 2
X_recovered, fl = recover_GRS_support(G2, F_list[0], F_list[1], F_list[j])
while fl == False:
    j += 1
    X_recovered, fl = recover_GRS_support(G2, F_list[0], F_list[1], F_list[j])
Y_recovered = recover_GRS_multiplier(G2, X_recovered, k)

In [6]:
print(X_recovered)
print(X_recovered)

[0, 1, 88, 24, 2, 78, 10, 121, 107, 38, 74, 48, 103, 39, 50, 101, 3, 128, 32, 73, 41, 35, 25, 130, 66, 123, 82, 59, 18, 31, 87, 77, 84, 17, 80, 14, 127, 43, 5, 58, 28, 56, 75, 37, 7, 12, 71, 126, 68, 116, 112, 62, 65, 97, 89, 64, 46, 111, 57, 113, 120, 115, 110, 114, 4, 52, 122, 81, 42, 8, 60, 102, 15, 92, 72, 55, 91, 34, 109, 23, 47, 90, 118, 63, 98, 117, 106, 99, 61, 36, 76, 29, 19, 13, 51, 108, 26, 96, 53, 69, 100, 54, 30, 79, 85, 33, 129]
[0, 1, 88, 24, 2, 78, 10, 121, 107, 38, 74, 48, 103, 39, 50, 101, 3, 128, 32, 73, 41, 35, 25, 130, 66, 123, 82, 59, 18, 31, 87, 77, 84, 17, 80, 14, 127, 43, 5, 58, 28, 56, 75, 37, 7, 12, 71, 126, 68, 116, 112, 62, 65, 97, 89, 64, 46, 111, 57, 113, 120, 115, 110, 114, 4, 52, 122, 81, 42, 8, 60, 102, 15, 92, 72, 55, 91, 34, 109, 23, 47, 90, 118, 63, 98, 117, 106, 99, 61, 36, 76, 29, 19, 13, 51, 108, 26, 96, 53, 69, 100, 54, 30, 79, 85, 33, 129]


In [7]:
C_rec = codes.GeneralizedReedSolomonCode(X_recovered, k, Y_recovered)
S = C_rec.generator_matrix().solve_left(G2)

In [8]:
s = y_enc * Hpub.T
# let's check that s == e * (H1.T)
s == e * (H1.T)

True

In [9]:
# decoding
yy = (H1.T).solve_left(s)
yy_punct = vector(F, [yy[i] for i in range(n) if i not in punctured_indices])
e_rec = yy - (C_rec.decode_to_message(yy_punct) * S^-1) * G1
e_rec

(0, 0, 0, 0, 0, 0, 110, 0, 0, 0, 0, 17, 0, 0, 74, 96, 0, 0, 0, 0, 0, 0, 0, 0, 0, 102, 0, 0, 0, 0, 0, 0, 0, 0, 61, 0, 0, 0, 72, 0, 0, 0, 0, 0, 0, 53, 0, 0, 0, 47, 0, 0, 0, 0, 0, 0, 0, 84, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 43, 0, 0, 0, 0, 0, 0, 0, 107, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 36, 0, 0, 0, 126, 0, 0, 19, 0, 0, 0, 0, 0, 0, 0, 128, 0, 0, 0, 0, 0, 115, 0, 0, 0, 0, 0, 0, 118, 1, 0, 0)

In [10]:
e_rec == e

True

In [11]:
msg_rec = Gpub.solve_left(y_enc - e_rec*Epub)
msg_rec == msg

True